# Misclassification Examples for DistillBERT

In [1]:
import pandas as pd
import numpy as np
import re

from torch import nn
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import *
import torchvision.models as models


import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from torch.nn import Transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf


import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap

/home/vscode/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-19 15:17:17.663312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 15:17:22.846209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from text_process import process_text
training_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/train.csv')
val_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/val.csv')
testing_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/test.csv')
training_dataset, val_dataset, testing_dataset = process_text(training_dataset, val_dataset, testing_dataset)

In [3]:
dist_model = SentenceTransformer('stsb-distilbert-base')
dist_model.max_seq_length = 128

train_titles = training_dataset.proc_title.tolist()
train_embed = dist_model.encode(train_titles, show_progress_bar=True, convert_to_tensor=True)

val_titles = val_dataset.proc_title.tolist()
val_embed = dist_model.encode(val_titles, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 172/172 [00:15<00:00, 11.27it/s]


In [4]:
def get_textEmbeddings(model,text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    return text_embedding

In [5]:
text_embeddings={}
for index,row in  training_dataset.iterrows():
    txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
    imagePath = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'+row[1]
    text_embeddings[row[0]] = txt_emb

In [6]:
val_text_embeddings={}
for index,row in  val_dataset.iterrows():
    txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
    imagePath = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'+row[1]
    val_text_embeddings[row[0]] = txt_emb



In [9]:
X_train, X_val, y_train, y_val = text_embeddings, val_text_embeddings, training_dataset.label_group, val_dataset.label_group

# Convert X_train and X_val to pandas DataFrames
X_train_df = pd.DataFrame(list(X_train.values()))
X_val_df = pd.DataFrame(list(X_val.values()))

results = []

for k in [1, 2, 3, 5, 10, 25, 50, 100]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_df, y_train)
    y_pred = knn.predict(X_val_df)
    
    # Find misclassified data points
    misclassified_indices = np.where(y_pred != y_val)[0]
    misclassified_data = X_val_df.iloc[misclassified_indices]
    misclassified_labels = y_val.iloc[misclassified_indices]
    predicted_labels = y_pred[misclassified_indices]
    
    print(f"For k={k}, {len(misclassified_indices)} out of {len(y_val)} data points are misclassified.")
    
    for i in misclassified_indices:
        result = {'K': k, 'True label': y_val.iloc[i], 'Predicted label': y_pred[i]}
        results.append(result)
    
df_results = pd.DataFrame(results)


For k=1, 2905 out of 5480 data points are misclassified.
For k=2, 3523 out of 5480 data points are misclassified.
For k=3, 3752 out of 5480 data points are misclassified.
For k=5, 4028 out of 5480 data points are misclassified.
For k=10, 4243 out of 5480 data points are misclassified.
For k=25, 4517 out of 5480 data points are misclassified.
For k=50, 4705 out of 5480 data points are misclassified.
For k=100, 4849 out of 5480 data points are misclassified.


In [10]:
df_results

,K,True label,Predicted label
0,1,631365480,962113684
1,1,612013449,1783253855
2,1,3651485151,3173716369
3,1,663012346,762113799
4,1,1761007187,3319810322
...,...,...,...
32517,100,679648252,3222645954
32518,100,2595278679,452508504
32519,100,2206777473,1237550763
32520,100,3810846799,933146825


In [11]:
path_to_csv='/workspaces/Shopee-Price-Match-Guarantee/30_results/DistillBERT_missclassified.csv'

In [12]:
df_results.to_csv(path_to_csv, index=False)